In [ ]:
import pip

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
requirements="""
ale-py==0.7.4
AutoROM==0.4.2
AutoROM.accept-rom-license==0.4.2
certifi
charset-normalizer==3.1.0
click==8.1.3
cloudpickle==2.2.1
cmake==3.26.3
contourpy==1.0.7
cycler==0.11.0
Farama-Notifications==0.0.4
filelock==3.12.0
fonttools==4.39.3
gym==0.23.1
gym-notices==0.0.8
gymnasium-notices==0.0.1
idna==3.4
imageio-ffmpeg==0.4.8
importlib-metadata==6.5.0
importlib-resources==5.12.0
jax-jumpy==1.0.0
Jinja2==3.1.2
kiwisolver==1.4.4
lit==16.0.1
MarkupSafe==2.1.2
matplotlib==3.7.1
mpmath==1.3.0
networkx==3.1
numpy
opencv-python==4.6.0.66
packaging==23.1
pandas
Pillow==9.5.0
pyparsing==3.0.9
python-dateutil==2.8.2
pytz==2023.3
requests
six==1.16.0
stable-baselines3==1.2.0
sympy==1.11.1
torch
torchvision
tqdm==4.65.0
typing_extensions
tzdata==2023.3
urllib3==1.26.15
zipp==3.15.0
moviepy==1.0.3
"""

with open("requirements.txt", "w") as f:
  f.writelines(requirements)

# Since there are a number of old dependencies we need to fix this.
#pip.main(['uninstall', '--yes', '-r', 'requirements.txt'])
pip.main(['install', '-r', 'requirements.txt'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting ale-py==0.7.4 (from -r requirements.txt (line 2))

Downloading ale_py-0.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)

Output()

Collecting AutoROM==0.4.2 (from -r requirements.txt (line 3))

...

Found existing installation: importlib_metadata 7.1.0

Uninstalling importlib_metadata-7.1.0:

Successfully uninstalled importlib_metadata-7.1.0

Successfully installed AutoROM-0.4.2 AutoROM.accept-rom-license-0.4.2 Farama-Notifications-0.0.4 Jinja2-3.1.2 MarkupSafe-2.1.2 Pillow-9.5.0 ale-py-0.7.4 charset-normalizer-3.1.0 click-8.1.3 cmake-3.26.3 contourpy-1.0.7 cycler-0.11.0 filelock-3.12.0 fonttools-4.39.3 gym-0.23.1 gymnasium-notices-0.0.1 idna-3.4 imageio-ffmpeg-0.4.8 importlib-metadata-6.5.0 importlib-resources-5.12.0 jax-jumpy-1.0.0 kiwisolver-1.4.4 lit-16.0.1 networkx-3.1 nvidia-cublas-cu12-12.1.3.1 nvidia-cuda-cupti-cu12-12.1.105 nvidia-cuda-nvrtc-cu12-12.1.105 nvidia-cuda-runtime-cu12-12.1.105 nvidia-cudnn-cu12-8.9.2.26 nvidia-cufft-cu12-11.0.2.54 nvidia-curand-cu12-10.3.2.106 nvidia-cusolver-cu12-11.4.5.107 nvidia-cusparse-cu12-12.1.0.106 nvidia-nccl-cu12-2.19.3 nvidia-nvjitlink-cu12-12.4.127 nvidia-nvtx-cu12-12.1.105 opencv-python-4.6.0.66 packaging-23.1 pyparsing-3.0.9 pytz-2023.3 stable-baselines3-1.2.0 sympy-1.11.1 tqdm-4.65.0 tzdata-2023.3 urllib3-1.26.15 zipp-3.15.0

0

In [ ]:
# These are the recommended hyperparameters
class Hyperparameters:
    env_id = "BreakoutNoFrameskip-v4"
    exp_name = "DQN_Breakout"
    seed = 1
    torch_deterministic = True
    capture_video = True
    save_model = True

    total_timesteps = 1_000_000
    learning_rate = 1e-4
    buffer_size = 1_000_000
    gamma = 0.99
    tau = 1
    target_network_frequency = 1000
    batch_size = 32
    start_e = 1
    end_e = 0.01
    exploration_fraction = 0.1
    learning_starts = 80_000
    train_frequency = 4


In [ ]:
import os
import random
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# FIXES
import tqdm
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

# NOT RECOMMENDED FIX...
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Imports all our hyperparameters from the other file
#from hyperparams import Hyperparameters as params
params = Hyperparameters

# stable_baselines3 have wrappers that simplifies
# the preprocessing a lot, read more about them here:
# https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
)
from stable_baselines3.common.buffers import ReplayBuffer


# Creates our gym environment and with all our wrappers.
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk

class QNetwork(nn.Module):
    def __init__(self, env):
        super(QNetwork, self).__init__()
        self.network = nn.Sequential(
            # First convolutional layer with ReLU
            nn.Conv2d(4, 16, kernel_size=8, stride=4),
            nn.ReLU(),
            # Second convolutional layer with ReLU
            nn.Conv2d(16, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            # Flattening layer, using nn.Flatten to handle batch flattening
            nn.Flatten(),
            # Fully-connected layer with ReLU
            nn.Linear(32 * 9 * 9, 256),
            nn.ReLU(),
            # Output layer, no activation function here
            nn.Linear(256, envs.single_action_space.n)
        )

    def forward(self, x):
        # Normalize input by dividing by 255.0, as pixel values range from 0 to 255
        x = x / 255.0
        return self.network(x)



def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

if __name__ == "__main__":
    run_name = f"{params.env_id}__{params.exp_name}__{params.seed}__{int(time.time())}"

    random.seed(params.seed)
    np.random.seed(params.seed)
    torch.manual_seed(params.seed)
    torch.backends.cudnn.deterministic = params.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # env setup
    envs = gym.vector.SyncVectorEnv([make_env(params.env_id, params.seed, 0, params.capture_video, run_name)])
    assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

    q_network = QNetwork(envs).to(device)
    optimizer = optim.Adam(q_network.parameters(), lr=params.learning_rate)
    target_network = QNetwork(envs).to(device)
    target_network.load_state_dict(q_network.state_dict())

    # We’ll be using experience replay memory for training our DQN.
    # It stores the transitions that the agent observes, allowing us to reuse this data later.
    # By sampling from it randomly, the transitions that build up a batch are decorrelated.
    # It has been shown that this greatly stabilizes and improves the DQN training procedure.
    rb = ReplayBuffer(
        params.buffer_size,
        envs.single_observation_space,
        envs.single_action_space,
        device,
        optimize_memory_usage=False,
        handle_timeout_termination=True,
    )

    loss = None
    obs = envs.reset()
    progress = tqdm.tqdm(range(params.total_timesteps))
    for global_step in progress:
        # Here we get epsilon for our epislon greedy.
        epsilon = linear_schedule(params.start_e, params.end_e, params.exploration_fraction * params.total_timesteps, global_step)

        if random.random() < epsilon:
            actions = envs.action_space.sample()
        else:
            q_values = q_network(torch.tensor(obs, dtype=torch.float32, device=device))
            actions = torch.argmax(q_values, dim=1).cpu().numpy()

        # Take a step in the environment
        next_obs, rewards, dones, infos = envs.step(actions)

        # Here we print our reward.
        for info in infos:
            if "episode" in info.keys():
                progress.set_description(f"global_step={global_step}, episodic_return={info['episode']['r']}, last_loss={loss}")
                break

        # Save data to replay buffer
        real_next_obs = next_obs.copy()
        for idx, d in enumerate(dones):
            if d:
                real_next_obs[idx] = infos[idx]["terminal_observation"]

        # Here we store the transitions in D
        rb.add(obs, real_next_obs, actions, rewards, dones, infos)

        obs = next_obs
        # Training
        if global_step > params.learning_starts:
            if global_step % params.train_frequency == 0:
                # Sample random minibatch of transitions from D
                data = rb.sample(params.batch_size)
                # You can get data with:
                # data.observation, data.rewards, data.dones, data.actions

                with torch.no_grad():
                    # Now we calculate the y_j for non-terminal phi.
                    # Get the maximum Q-value for each next state
                    target_max, _ = q_network(data.next_observations).max(dim=1)
                    # Calculate y_j, taking into account whether the state is terminal
                    td_target = data.rewards + (1 - data.dones) * params.gamma * target_max

                old_val = q_network(data.observations).gather(1, data.actions).squeeze()
                loss = F.mse_loss(td_target, old_val)

                # perform our gradient decent step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network
            if global_step % params.target_network_frequency == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        params.tau * q_network_param.data + (1.0 - params.tau) * target_network_param.data
                    )

    if params.save_model:
        model_path = f"runs/{run_name}/{params.exp_name}_model"
        torch.save(q_network.state_dict(), model_path)
        print(f"model saved to {model_path}")

    envs.close()


global_step=7890, episodic_return=0.0, last_loss=None:   8%|▊         | 7980/100000 [00:17<03:27, 443.83it/s]<ipython-input-62-eaf89ab25824>:174: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(td_target, old_val)
global_step=10583, episodic_return=1.0, last_loss=0.0032598606776446104:  11%|█         | 10581/100000 [00:26<04:57, 300.11it/s]<ipython-input-62-eaf89ab25824>:174: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(td_target, old_val)
global_step=19720, episodic_return=2.0, last_loss=1.6861600897755125e-06:  20%|█▉        | 19714/100000 [00:58<04:34, 292.28it/s]<ipython-input-62-eaf89ab25824>:174: UserWarning: 

RuntimeError: Parent directory runs/BreakoutNoFrameskip-v4__DQN_Breakout__1__1715083038 does not exist.

In [ ]:
(data.rewards + (1 - data.dones)).shape

torch.Size([32, 1])

In [ ]:
torch.tensor(data.next_observations, dtype=torch.float32, device=device).max(dim=1,keepdim=True)


<ipython-input-53-db73c6934829>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data.next_observations, dtype=torch.float32, device=device).max(dim=1,keepdim=True)


torch.return_types.max(
values=tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,


In [ ]:
# If you are running this on a server you will not be able to use render_mode="human"
# instead use render_mode="rgb_array" since you cannot ouput to a display.

# WARNING: This may not run on M1 macs.
# If you really want to run the example, I recommend creating a seperate
# conda env and installing gym with pip install gym[atari, accept-rom-license]
# But use the ex6 environment when working on the dqn file.

import gym
import time

env = gym.make("ALE/Breakout-v5", render_mode="rgb_array")
observation, info = env.reset(seed=42)

for _ in range(1000):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()
print("Executed environment successfully.")
